- 문서 분류 모델 학습하기
  - https://ratsgo.github.io/nlpbook/docs/doc_cls/train/

- ratsnlp/ratsnlp/nlpbook/
  - https://github.com/ratsgo/ratsnlp/tree/master/ratsnlp/nlpbook

In [ ]:
# /usr/local/lib/python3.10/dist-packages/ratsnlp 에 설치
# /usr/local/lib/python3.10/dist-packages/ratsnlp -> 직접 찾아본 경로 확인!
!pip install ratsnlp

# 외부 라이브러리에서 설치할 수 있는것은..(사용자가 만듦.)
# https://pypi.org/ 여기에 올리면 파이썬에서 임포트해서 쓸 수 있음. 즉, 누구나 본인만의 라이브러리를 만들어서 올리고 임포트 가능

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.



학습 인자를 위한 데이터클래스: ClassificationTrainArguments

이 클래스는 분류 모델 학습과 관련된 다양한 설정을 보유하도록 설계되었습니다. dataclass의 사용은 __init__, __repr__ 등과 같은 내장 메서드를 포함한 클래스 인스턴스 생성을 단순화합니다.

속성:
- pretrained_model_name: 사용할 사전 훈련된 모델의 이름입니다.
- downstream_task_name: 조직 목적을 위한 작업의 이름입니다.
- downstream_corpus_name: 사용된 데이터의 구체적인 이름입니다.
- downstream_corpus_root_dir: 데이터가 저장된 루트 디렉토리입니다.
- downstream_model_dir: 학습된 모델이 저장될 디렉토리입니다.
- max_seq_length: 토큰화를 위한 최대 시퀀스 길이입니다.
- save_top_k: 모니터링 조건에 기반한 상위 모델 체크포인트를 저장할 개수입니다.
- monitor: 최고의 모델 체크포인트를 저장하기 위한 모니터링 조건입니다(예: 검증 손실).
- seed: 재현성을 위한 랜덤 시드입니다.
- overwrite_cache: 캐시된 학습 및 평가 세트를 덮어쓸지 여부입니다.
- force_download: 이미 존재하더라도 데이터 및 모델을 강제로 다운로드할지 여부입니다.
- test_mode: 테스트를 위한 빠른 개발 실행을 활성화합니다.
- learning_rate: 최적화를 위한 학습률입니다.
- epochs: 학습 에폭 수입니다.
- batch_size: 학습을 위한 배치 크기입니다. 0으로 설정하면 최적의 배치 크기가 자동으로 찾아집니다.
- cpu_workers: 데이터 로딩을 위한 CPU 워커의 수입니다.
- fp16: 더 빠른 계산을 위해 16비트 부동 소수점으로 학습을 활성화합니다.
- tpu_cores: 사용 가능한 경우 TPU 코어에서 학습을 활성화합니다.

- 이 속성들은 https://github.com/ratsgo/ratsnlp/blob/master/ratsnlp/nlpbook/classification/arguments.py 에 있음. ratsnlp의 arguments.py에서 확인 할 수 있음

In [ ]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls1",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=1,

    seed=7,
)
# https://github.com/ratsgo/ratsnlp/blob/master/ratsnlp/nlpbook/utils.py
# utils.py 226번째줄에 set_seed 있음. seed가 7로 넣으면 어떻게 되는지 코드 볼수 있음

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [ ]:
# args에 지정된 시드로 고정하는 역할
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 말뭉치 다운로드
- 실습에 사용할 말뭉치(Naver Sentiment Movie review Corpus)를 다운로드합니다.
- 코랩 환경 로컬의 root_dir(/content/Korpora) 이하에 저장
- /content/Korpora -> 직접 경로 찾아봄..

In [ ]:
from Korpora import Korpora
Korpora.fetch( # 정 한국어 코퍼스를 Korpora 서버에서 로컬로 다운로드하는 역할
    corpus_name=args.downstream_corpus_name, # 다운로드할 말뭉치의 이름을 지정
    root_dir=args.downstream_corpus_root_dir, # 말뭉치가 다운로드될 로컬 디렉토리의 경로를 지정
    force_download=True, # 동일한 이름의 코퍼스가 이미 존재하는 경우에도 코퍼스를 다시 다운로드
)
# 말뭉치 = corpus
# Korpora는 다양한 한국어 말뭉치를 쉽게 다운로드하고 사용할 수 있게 해주는 파이썬 패키지

[nsmc] download ratings_train.txt: 14.6MB [00:00, 63.3MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 23.7MB/s]                            


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다.
kcbert-base 모델이 사용하는 토크나이저(tokenizer)를 선언합니다. 토크나이저(tokenizer)는 토큰화를 수행하는 프로그램이라는 뜻

/usr/local/lib/python3.10/dist-packages/transformers

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name, # 이 이름은 BERT 모델의 특정 버전.. "bert-base-uncased", "bert-large-cased" 등이 될 수 있음
    do_lower_case=False, # 텍스트를 토큰화할 때 대소문자를 구분하여 처리
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

tokenizer = BertTokenizer.from_pretrained(...) 이 부분은 사실상 "미리 학습된 언어 규칙 책"을 가져와서 우리가 가진 텍스트를 컴퓨터가 이해할 수 있는 방식으로 바꾸는 일을 합니다.

예를 들어, 우리가 영어 문장을 읽을 때, 단어와 문장 부호를 구별해서 이해하듯이, 컴퓨터도 텍스트를 "이해"하기 위해 비슷한 과정을 거칩니다. 하지만 컴퓨터는 우리처럼 자연어를 직접 이해할 수 없기 때문에, 텍스트를 컴퓨터가 처리할 수 있는 형태, 즉 숫자나 기호의 나열로 바꿔야 합니다. 이 작업을 '토큰화(tokenization)'라고 합니다.

from_pretrained(...) 부분은 이 토큰화 작업을 수행하기 위한 규칙이나 방법을 담고 있는 "미리 학습된 토크나이저"를 불러옵니다. 이 미리 학습된 토크나이저는 특정 언어 모델(BERT 등)과 함께 사용되며, 해당 모델이 처음 학습될 때 사용된 규칙을 그대로 가져와서 우리의 텍스트에 적용합니다. 이렇게 하면 우리가 사용하는 텍스트가 모델이 처음 학습했던 텍스트와 같은 방식으로 처리되어, 모델이 텍스트를 더 잘 이해하고 예측할 수 있게 됩니다.

간단히 말해, BertTokenizer.from_pretrained(...)는 특정 BERT 모델을 위해 미리 설정된 언어 규칙을 불러와서 우리의 텍스트를 모델이 "읽을 수 있는" 형태로 바꿔주는 도구라고 할 수 있습니다.

## 학습데이터 구축
학습데이터를 만듭니다.

- 딥러닝 모델을 학습하려면 학습데이터를 배치(batch) 단위로 지속적으로 모델에 공급해 주어야 합니다. 파이토치(PyTorch)에서는 이 역할을 데이터 로더(DataLoader)가 수행
- NsmcCorpus는 CSV 파일 형식의 NSMC 데이터를 “문장(영화 리뷰) + 레이블(긍정, 부정)” 형태로 읽어들이는 역할
- ClassificationDataset는 각 instance를 포함하는 데이터셋 역할을 수행.
ClassificationDataset 클래스를 좀 더 자세히 살펴보겠습니다. 이 클래스는 NsmcCorpus와 토크나이저를 품고 있는데요. NsmcCorpus가 넘겨준 데이터(문장, 레이블)를 모델이 학습할 수 있는 형태로 가공합니다. 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수(integer)로 바꿔주는 역할을 한다.
- ClassificationDataset은 input_ids, attention_mask, token_type_ids의 길이가 모두 128인 이유는 토큰 기준 최대 길이(max_seq_length)를 args에서 128로 설정해 두었기 때문입니다.
- input_ids에 패딩 토큰([PAD])의 인덱스에 해당하는 0이 많이 붙어 있음을 확인할 수 있습니다. 분석 대상 문장의 토큰 길이가 max_seq_length보다 짧아서입니다. 이보다 긴 문장일 경우 128로 줄입니다.
- attention_mask는 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타내며 token_type_ids는 세그먼트(segment) 정보로 기본값은 모두 0으로 넣습니다. label은 정수로 변환
- sampler는 샘플링 방식을 정의합니다. 데이터 로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼을 비복원(replacement=False) 랜덤 추출한다. 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용한다. SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 한다. 학습 때 배치 구성은 랜덤으로 하는 것이 좋으나 평가할 때는 평가용 데이터 전체를 사용하기 때문에 굳이 랜덤으로 구성할 이유가 없기 때문에 SequentialSampler를 쓴다.
- collate_fn은 이렇게 뽑힌 인스턴스를 배치로 만드는 역할을 하는 함수


In [ ]:
# 이 코드는 PyTorch 라이브러리를 사용하여
# 한국어 영화 리뷰 분류(Naver Sentiment Movie Corpus, NSMC) 작업을 위한 데이터 로더를 설정하는 과정
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler #  PyTorch에서 데이터를 배치 단위로 모델에 공급하기 위해 사용되는 클래스
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset #  NSMC 데이터를 처리하고 분류 작업을 위한 데이터셋을 생성하는 데 사용
corpus = NsmcCorpus() #  데이터 예제를 추출하고 레이블을 관리하는 메소드를 제공
# NsmcCorpus(): NSMC 데이터셋의 인스턴스를 생성합니다.
# 이 클래스는 데이터를 로드하고, 각 리뷰의 텍스트와 해당 리뷰가 긍정적인지 부정적인지를 나타내는 레이블을 관리
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

train_dataloader = DataLoader( # 훈련 데이터셋을 위한 데이터 로더를 생성. 이 데이터 로더는 모델 학습 시 배치 단위로 데이터를 모델에 공급하는 역할
    train_dataset,
    batch_size=args.batch_size, # 각 배치의 크기를 지정합니다. args.batch_size는 이 크기를 저장하는 변수
    sampler=RandomSampler(train_dataset, replacement=False), # 샘플링시 중복을 허용하지 않아 미니배치는 고유한 샘플로만 구성
    collate_fn=nlpbook.data_collator, # 배치 내의 데이터를 어떻게 결합할지 정의하는 함수. nlpbook.data_collator는 이를 위한 함수를 참조
    drop_last=False, # 배치 크키로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers, # 데이터 로딩을 위해 사용할 프로세스의 수를 지정. args.cpu_workers는 이 수를 저장하는 변수
)

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [ ]:
# ClassificationDataset 클래스를 사용하여 분류 작업을 위한 검증 데이터셋을 생성합니다. 이 데이터셋은 모델 평가를 위해 사용
# 검증 데이터셋 설정
val_dataset = ClassificationDataset(
    args=args, # 이전에 정의된 설정을 포함하는 객체. 데이터셋 경로, 토크나이저 설정, 배치 크기 등의 정보가 포함.
    corpus=corpus, # NsmcCorpus 인스턴스로, 실제 데이터와 레이블을 관리.
    tokenizer=tokenizer, # 텍스트 데이터를 토큰화하기 위해 사용되는 토크나이저
    mode="test", # 이 파라미터는 검증(또는 테스트) 데이터셋을 생성하기 위해 사용.
    # 'test' 모드에서는 모델이 이미 본 훈련 데이터가 아닌 새로운 데이터에 대한 예측 성능을 평가
)

# 검증 데이터 로더 설정
val_dataloader = DataLoader( # 데이터 로더는 모델 평가 시 검증 데이터를 배치 단위로 모델에 공급하는 역할
    val_dataset,
    batch_size=args.batch_size, # 각 배치의 크기를 지정합니다. 이는 모델에 한 번에 공급되는 데이터 샘플의 수를 결정
    sampler=SequentialSampler(val_dataset), # SequentialSampler는 데이터셋에서 샘플을 순차적으로 선택하여 배치를 구성.
    # 검증 과정에서는 데이터의 순서를 유지하는 것이 일반적이므로, 무작위 샘플링 대신 순차 샘플링이 사용
    collate_fn=nlpbook.data_collator, # nlpbook.data_collator는 배치 데이터를 적절한 형태로 결합하기 위한 함수를 참조
    drop_last=False, #  마지막 배치의 크기가 batch_size보다 작더라도 사용하겠다는 의미입니다
    # 검증 과정에서는 모든 데이터를 사용하는 것이 중요할 수 있으므로, 마지막 불완전한 배치도 평가에 포함됩니다.
    num_workers=args.cpu_workers,
)


# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화
- BertForSequenceClassification은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스

In [ ]:
# 이 코드는 Hugging Face의 transformers 라이브러리를 사용하여
# 사전 훈련된 BERT 모델을 기반으로 한 시퀀스 분류(sequence classification) 모델을 설정하는 과정
# 먼저 config 만들기..

# pretrained_model_name="beomi/kcbert-base",
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels, # 분류 작업에 사용될 레이블의 총 수를 지정합니다.
    # corpus.num_labels는 데이터셋(corpus)에 정의된 고유한 레이블의 수를 나타냅니다.
    # 예를 들어, 긍정/부정 두 가지 레이블로 구성된 이진 분류 문제에서는 num_labels가 2가 됩니다.
)

In [ ]:
# config 만든거 넣기..
model = BertForSequenceClassification.from_pretrained( # 사전 훈련된 BERT 모델을 기반으로 시퀀스 분류 작업을 위한 모델을 초기화하고 생성하는 과정
        args.pretrained_model_name,
        config=pretrained_model_config, # config 인자에는 사전 훈련된 모델을 사용자 정의 설정으로 초기화하기 위한 구성 객체를 전달.
        # pretrained_model_config는 이전 단계에서 생성한 BertConfig 인스턴스.
        #  이 구성 객체는 모델의 레이어 수, 은닉 유닛 수, 레이블 수 등의 세부 사항을 포함하며, 특히 여기서는 분류 작업을 위한 레이블 수(num_labels)가 중요
)
# 이렇게 하면 사전 훈련된 가중치를 유지하면서도, 특정 작업(여기서는 시퀀스 분류)에 맞게 모델을 조정할 수 있음.

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

# 학습 준비
Task와 Trainer를 준비합니다.

In [ ]:
# https://github.com/ratsgo/ratsnlp/blob/master/ratsnlp/nlpbook/classification/task.py
# task.py에  validation_step 코드가 있음.
# 텍스트 분류 작업을 설정하는 과정.
# ClassificationTask 클래스는 모델, 훈련 매개변수, 옵티마이저, 손실 함수 등 텍스트 분류 작업에 필요한 모든 구성 요소를 캡슐화함.
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args) # 텍스트 분류 작업을 위한 인스턴스를 생성.
# model: 이전 단계에서 초기화된 시퀀스 분류를 위한 모델 객체. BertForSequenceClassification을 사용한 모델이 여기에 해당
# 이 모델은 텍스트 입력을 받아 각 클래스에 대한 예측을 수행하는 역할
# args: 이 인자는 모델 훈련과 관련된 설정을 포함하는 객체. 일반적으로 배치 크기, 학습률, 훈련 에포크 수, 옵티마이저 선택 등과 같은 훈련 매개변수를 지정.

In [ ]:
trainer = nlpbook.get_trainer(args)
# get_trainer 함수는 args에 지정된 설정을 사용하여 모델의 훈련을 관리할 트레이너 객체를 초기화하고 생성.
# args: 이는 훈련 과정에서 사용될 다양한 매개변수를 포함하는 객체.
# 학습률, 배치 크기, 에포크 수, 사용할 디바이스(CPU 또는 GPU) 설정 등 훈련에 필요한 설정이 포함될 수 있음.

# 생성된 trainer 객체는 모델을 훈련시키는 데 사용됩니다.
# 이 객체는 훈련 데이터를 모델에 공급하고, 각 에포크마다 모델을 업데이트하며,
# 필요한 경우 검증 데이터셋을 사용하여 모델의 성능을 평가하는 등의 작업을 자동화합니다.
# trainer를 사용하면 훈련 과정을 더 쉽게 관리하고, 모델 훈련의 효율성과 재현성을 높일 수 있습니다

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-doccls`)에 저장됩니다.

In [ ]:
# 이 코드는 trainer 객체의 fit 메서드를 사용하여 모델 훈련을 시작하는 과정.
trainer.fit(
    task,
    train_dataloaders=train_dataloader, # 훈련 데이터셋을 배치 단위로 모델에 공급하기 위해 사용되는 데이터 로더입니다. 이 데이터 로더는 모델 훈련을 위한 주요 입력 데이터 소스 역할
    val_dataloaders=val_dataloader, #  검증 데이터셋을 모델에 공급하는 데 사용됩니다. 훈련 과정에서 정기적으로 모델의 성능을 검증 데이터셋을 사용하여 평가하며, 이를 통해 모델의 일반화 능력을 확인
)
# 95/50000 [10:13<89:30:01,  6.46s/it, loss=0.597, v_num=0, acc=0.500] -> 89시간 예상... 중단함.
# CPU로 하면 너무 오래걸리고 GPU 써야하는데 제한됨.. CPU로 하면 batch가 안먹을 수 있어서 아마 개수도 50000개로 뜨고 시간도 오래걸리는것임
# GPU로 한 강사님 실행 결과를 보면, 6251개로 뜨고 17분 걸림. 에폭은 같은 1로 설정.
# 그리고 강사님은 5.85it/s 인데, 초당 5.85 돈다는 뜻이고..
# 나는 6.46s/it 인데, 6.46초에 한번 돈다는 뜻임.... 엄청난 속도 차이..

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
